In [33]:
import re
import torch
import pandas as pd
import glob
import PIL.Image
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import os
import cv2
import torch.nn as nn
import torch.optim as optim
import datetime

In [34]:
FILES_PATH='/home/kostas/Autonomous_car/files/'

track_name = 'track5'
PARENT_PATH= FILES_PATH + track_name + '/' + 'direction1_50'
speed_value = 75
data_dirs=['direction1_75','direction2_75']
class_names = ['forward', 'stop', 'forward_left', 'forward_right']


In [35]:
class Directory:
    def __init__(self, parent_path):
        self.parent_path = parent_path

    def getSubdirectories(self):
        return [dI for dI in os.listdir(self.parent_path) if os.path.isdir(os.path.join(self.parent_path,dI))]

In [36]:
class ImageLoader:
    def __init__(self, parent_dir_path=''):
        self.parent_dir_path = parent_dir_path

    def load_image_paths(self):
        return glob.glob(self.parent_dir_path + '/*.jpg')


    def load_image(self, img_path, resize=True):

        img = PIL.Image.open(img_path)
        if resize:
            img = transforms.functional.resize(img, (224, 224))

        img = transforms.functional.to_tensor(img)
        img = transforms.functional.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return img

    def transformImage(self, image):
        image = transforms.functional.resize(image, (480, 640))
        image = transforms.functional.to_tensor(image)
        return image

In [37]:
class CsvLoader:
    def __init__(self, parent_dir_path=''):
        self.parent_dir_path = parent_dir_path

    def load_csv_path(self):
        return glob.glob(self.parent_dir_path + '/*.csv')

    def load_csv_to_df(self, csv_path):
        return pd.read_csv(csv_path, delimiter=',')

    def map_class_to_int(self, df):
        idx_to_label = {
            'forward':0 , 'stop':1 , 'forward_left':2, 'forward_right':3
        }
        return df.replace({'Move': idx_to_label})

In [38]:
class TrainTestDataSplitter:
    def __init__(self, train_batch_size=4, test_batch_size=1):
        self.train_batch_size = train_batch_size
        self.test_batch_size = test_batch_size

    def split_images(self, images_tensor):
        train_images = []
        test_images = []
        size = len(images_tensor)
        for i in range(size):
            if i % 5 == 0:
                test_images.append(images_tensor[i])
            else:
                train_images.append(images_tensor[i])
        print(len(train_images), len(test_images))
        return train_images, test_images

    def split_classes(self, output_classes_torch):
        train_classes = []
        test_classes = []
        size = len(output_classes_torch)
        for i in range(size):
            if i % 5 == 0:
                test_classes.append(output_classes_torch[i])
            else:
                train_classes.append(output_classes_torch[i])
        print(len(train_classes), len(test_classes))
        return train_classes, test_classes

In [39]:
parent_directory = Directory(PARENT_PATH)

subdirectories = parent_directory.getSubdirectories()
print(sorted(subdirectories))

all_image_paths= []
all_csv_paths = []
for subdirectory in sorted(subdirectories):
    subdirectory_path = PARENT_PATH + '/' + subdirectory
    imageLoader = ImageLoader(subdirectory_path)
    image_paths = imageLoader.load_image_paths()
    image_paths.sort(key=lambda f: int(re.sub('\D', '', f)))
    all_image_paths += image_paths

    csvLoader = CsvLoader(subdirectory_path)
    csvPath = csvLoader.load_csv_path()
    all_csv_paths += csvPath


['2021-01-02_14-29-02', '2021-01-02_14-29-48', '2021-01-02_14-31-03', '2021-01-02_14-31-43', '2021-01-02_14-32-22', '2021-01-02_14-32-50', '2021-01-02_14-34-54', '2021-01-02_14-36-34', '2021-01-02_14-37-08', '2021-01-02_14-37-46', '2021-01-02_14-38-23', '2021-01-02_14-39-06', '2021-01-02_14-40-18']


In [40]:
print(len(all_image_paths))
print(len(all_csv_paths))


5390
13


In [41]:
im = ImageLoader()
images =  [torch.tensor(im.load_image(path)) for path in all_image_paths]

print(len(images))


/home/kostas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


5390


In [42]:
cl= CsvLoader()

output_classes = np.zeros(0)
for path in all_csv_paths:
    csv = cl.load_csv_to_df(path)
    data_classes = cl.map_class_to_int(csv)['Move'] # .astype('int64')
    output_classes = np.concatenate((output_classes,data_classes), axis=0)

print(output_classes)

[1. 1. 1. ... 1. 1. 1.]


In [43]:

output_classes_tensor = torch.tensor(output_classes,  dtype=torch.int64)

target_onehot = torch.zeros(output_classes_tensor.shape[0], 4)
target_onehot.scatter_(1, output_classes_tensor.unsqueeze(1), 1.0)


tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [44]:
splitter = TrainTestDataSplitter(4,1)
train_images, test_images = splitter.split_images(images)
train_classes, test_classes = splitter.split_classes(output_classes)


4312 1078
4312 1078


In [45]:
train_data = [(train_images[i], train_classes[i]) for i in range(len(train_images))]
test_data = [(test_images[i], test_classes[i]) for i in range(len(test_images))]
train_data[10]

(tensor([[[ 0.9132,  0.8618,  0.8447,  ..., -0.1999, -0.1486, -0.1314],
          [ 0.8961,  0.8447,  0.7933,  ..., -0.1828, -0.1486, -0.1657],
          [ 0.8789,  0.8276,  0.7933,  ..., -0.1314, -0.1314, -0.1657],
          ...,
          [ 0.3994,  0.3823,  0.3652,  ..., -1.5185, -1.5014, -1.4500],
          [ 0.3652,  0.3823,  0.3823,  ..., -1.5014, -1.5185, -1.4500],
          [ 0.3994,  0.4166,  0.4337,  ..., -1.4329, -1.4500, -1.3987]],
 
         [[-0.3375, -0.3375, -0.3375,  ..., -0.8803, -0.8803, -0.8627],
          [-0.3550, -0.3550, -0.3375,  ..., -0.8627, -0.8803, -0.8978],
          [-0.3200, -0.3550, -0.3375,  ..., -0.8102, -0.8627, -0.8978],
          ...,
          [-0.5476, -0.5476, -0.5301,  ..., -2.0357, -2.0182, -1.9657],
          [-0.5651, -0.5476, -0.5126,  ..., -2.0357, -2.0357, -1.9832],
          [-0.5301, -0.5126, -0.4601,  ..., -1.9657, -1.9832, -1.9307]],
 
         [[ 0.0779,  0.0605,  0.0953,  ..., -0.6193, -0.6018, -0.5844],
          [ 0.0605,  0.0431,

In [46]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=0)
test_loader =  torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=True, num_workers=0)

In [47]:
model = models.resnet18(pretrained=True)

In [48]:
model.fc = torch.nn.Linear(512, len(class_names))
device = torch.device('cuda')
model = model.to(device)
# optimizer = optim.SGD(model.parameters(), lr=1e-2)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [49]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device, dtype=torch.int64)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        # if epoch == 1 or epoch % 5 == 0:
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [50]:
training_loop(
    n_epochs = 20,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2021-01-02 15:44:14.314589 Epoch 1, Training loss 0.5366479787985917
2021-01-02 15:54:15.955117 Epoch 2, Training loss 0.4029280909782894
2021-01-02 16:04:29.464301 Epoch 3, Training loss 0.3486081827747254
2021-01-02 16:14:47.151083 Epoch 4, Training loss 0.31464093311988717
2021-01-02 16:25:19.648729 Epoch 5, Training loss 0.29071583991942856
2021-01-02 16:35:52.536326 Epoch 6, Training loss 0.26312967361538914
2021-01-02 16:46:26.550088 Epoch 7, Training loss 0.2538468446265059
2021-01-02 16:57:02.578911 Epoch 8, Training loss 0.219572668430652
2021-01-02 17:07:39.867892 Epoch 9, Training loss 0.1917830487576716
2021-01-02 17:18:18.191027 Epoch 10, Training loss 0.16649571656264475
2021-01-02 17:28:58.433241 Epoch 11, Training loss 0.1508078682792532
2021-01-02 17:39:34.980029 Epoch 12, Training loss 0.12861417167778622
2021-01-02 17:50:07.721597 Epoch 13, Training loss 0.11238504408452181
2021-01-02 18:00:40.207945 Epoch 14, Training loss 0.09716991763664286
2021-01-02 18:11:09.537

In [51]:
torch.save(model.state_dict(), FILES_PATH + 'track5_1_model.pt')
